In [2]:
from dotenv import load_dotenv
import sys

sys.path.append("../common")
load_dotenv()

True

In [3]:
import os
from langsmith_tracker import set_tracking
from langchain_print import stream_response

# 인스턴스를 생성할 때 필요한 매개변수를 전달합니다.
set_tracking(project_name="03.OutputParser")

Langsmith 추적이 활성화되었습니다. [프로젝트명: 03.OutputParser]


### PydandicOutputParser
  
`PydanticOutputParser`는 Pydantic 모델을 기반으로 **LLM(Language Learning Model)**의 출력을 구조화된 Python 객체로 변환하는 클래스입니다.  
이 클래스는 LLM의 자연어 처리 결과를 사용자가 필요로 하는 명확하고 체계적인 형태(스키마)로 변환합니다. 특히, JSON과 같은 구조화된 형식을 처리하거나 특정 필드와 조건을 강제해야 할 때 유용하게 활용됩니다.
  
`PydanticOutputParser` 에는 두 가지 핵심 메서드가 구현 되어야 합니다.  

1. `parse()`: LLM의 출력 문자열을 받아 Pydantic 모델 인스턴스로 변환합니다.
2. `get_format_instructions()`: LLM이 출력해야 할 데이터의 형식을 정의하는 지침을 제공합니다.
  
데이터의 일관성과 유효성을 보장하는 도구입니다.
  
Pydantic 공식 문서 : https://docs.pydantic.dev/latest/    

In [5]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [6]:
email_conversation = """
From: siverempty@blockchainproject.com (박은빈, 기획팀)
To: terry@blockchainproject.com (김태리, 개발팀)
Subject: [수정 요청] 스마트 컨트랙트 민팅 조건 수정 필요


안녕하세요, 태리님.
기획팀 박은빈입니다.

현재 진행 중인 블록체인 익스플로러와 관련하여, 민팅 조건에 따른 문제점이 확인되어 수정 요청드립니다.

1. 발생한 상황
현재 스마트 컨트랙트에서는 CRO 가격이 $0.75 이상일 때 민팅이 가능하도록 설정되어 있습니다.
문제는 TWAP(시간 가중 평균 가격) 기준으로 민팅 조건이 적용되면서, 실제 시장 가격이 조건을 충족함에도 불구하고 TWAP 값이 실시간 가격보다 낮아 민팅이 차단되는 사례가 발생하고 있습니다.

특히, 시장 변동성이 큰 경우 TWAP이 실시간 가격을 제대로 반영하지 못하는 문제가 자주 보고되었습니다.

2. 문제의 영향
- 사용자 불만 증가: 사용자들이 민팅 오류를 겪고 VOC(고객 문의)가 급증하고 있습니다.
- 신뢰도 저하: 프로젝트의 신뢰도에 부정적인 영향을 미칠 가능성이 높습니다.
- 활성화 저해: 민팅 활성화 목표와 초기 사용자 유입 계획에 차질이 예상됩니다.

3. 요청드리는 수정 사항
- TWAP 기준 개선:
  - TWAP과 실시간 가격 차이를 줄이는 보정 로직 추가.
  - TWAP 기준을 사용하는 대신 실시간 가격 기반 조건으로 변경 검토.
- 허용 오차 범위 적용:
  - TWAP 값이 실시간 가격과 일정 범위 내에서 차이가 있을 경우 민팅을 허용하는 유연성 확보.
- 테스트 및 검증:
  - 수정된 로직에 대한 충분한 QA와 시뮬레이션 테스트를 진행해 재발 방지 확인.

4. 참고 자료
- 문제 발생 시간: 2024년 11월 18일 오후 3시경.
- 관련 로그: 첨부파일 [minting_error_log.txt]
- 사용자 문의 사례: 사용자 VOC 데이터 제공 가능 (요청 시 전달).

민팅 활성화와 사용자 경험 개선을 위해 긴급히 처리해주시면 감사하겠습니다. 필요하신 추가 자료가 있으면 언제든 말씀 부탁드립니다.

감사합니다.
박은빈
기획팀
Email: siverempty@blockchainproject.com
Phone: 010-1234-5678
"""

#### 출력 파서를 사용하지 않는 경우

In [7]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from itertools import chain

prompt = PromptTemplate.from_template(
    "다음의 이메일 내용중 중요한 내용을 추출해주세요.\n\n{email_conversation}"
)

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
chain = prompt | llm
answer = chain.stream({"email_conversation": email_conversation})
output = stream_response(answer, return_output=True)

이메일의 중요한 내용은 다음과 같습니다:

1. **발생한 상황**: 
   - 현재 스마트 컨트랙트에서 CRO 가격이 $0.75 이상일 때만 민팅이 가능하도록 설정되어 있음.
   - TWAP(시간 가중 평균 가격) 기준으로 민팅 조건이 적용되어, 시장 가격이 조건을 충족하더라도 TWAP 값이 낮아 민팅이 차단되는 사례 발생.
   - 시장 변동성이 클 경우 TWAP이 실시간 가격을 제대로 반영하지 못하는 문제.

2. **문제의 영향**:
   - 사용자 불만 증가 및 VOC(고객 문의) 급증.
   - 프로젝트 신뢰도 저하 가능성.
   - 민팅 활성화 목표와 초기 사용자 유입 계획에 차질 예상.

3. **요청드리는 수정 사항**:
   - TWAP 기준 개선: TWAP과 실시간 가격 차이를 줄이는 보정 로직 추가 및 실시간 가격 기반 조건으로 변경 검토.
   - 허용 오차 범위 적용: TWAP 값과 실시간 가격 간 차이가 일정 범위 내일 경우 민팅 허용.
   - 테스트 및 검증: 수정된 로직에 대한 QA와 시뮬레이션 테스트 진행.

4. **참고 자료**:
   - 문제 발생 시간: 2024년 11월 18일 오후 3시경.
   - 관련 로그: 첨부파일 [minting_error_log.txt].
   - 사용자 문의 사례: VOC 데이터 제공 가능 (요청 시 전달).

5. **긴급 요청**: 민팅 활성화와 사용자 경험 개선을 위해 긴급히 처리 요청.

In [8]:
print(output)

이메일의 중요한 내용은 다음과 같습니다:

1. **발생한 상황**: 
   - 현재 스마트 컨트랙트에서 CRO 가격이 $0.75 이상일 때만 민팅이 가능하도록 설정되어 있음.
   - TWAP(시간 가중 평균 가격) 기준으로 민팅 조건이 적용되어, 시장 가격이 조건을 충족하더라도 TWAP 값이 낮아 민팅이 차단되는 사례 발생.
   - 시장 변동성이 클 경우 TWAP이 실시간 가격을 제대로 반영하지 못하는 문제.

2. **문제의 영향**:
   - 사용자 불만 증가 및 VOC(고객 문의) 급증.
   - 프로젝트 신뢰도 저하 가능성.
   - 민팅 활성화 목표와 초기 사용자 유입 계획에 차질 예상.

3. **요청드리는 수정 사항**:
   - TWAP 기준 개선: TWAP과 실시간 가격 차이를 줄이는 보정 로직 추가 및 실시간 가격 기반 조건으로 변경 검토.
   - 허용 오차 범위 적용: TWAP 값과 실시간 가격 간 차이가 일정 범위 내일 경우 민팅 허용.
   - 테스트 및 검증: 수정된 로직에 대한 QA와 시뮬레이션 테스트 진행.

4. **참고 자료**:
   - 문제 발생 시간: 2024년 11월 18일 오후 3시경.
   - 관련 로그: 첨부파일 [minting_error_log.txt].
   - 사용자 문의 사례: VOC 데이터 제공 가능 (요청 시 전달).

5. **긴급 요청**: 민팅 활성화와 사용자 경험 개선을 위해 긴급히 처리 요청.


출력파서를 사용하지 않았을때, 
- 위의 답변에서 만약에 발신자 또는 수신자의 정보를 포함한 내용을 받으려면 프롬프트를 수정해야 합니다.
- 답변이 문자열이기 때문에 원하는 정보를 추출하는 것이 쉽지 않습니다.

위와 같은 문제들을 해결하기 위해 아래의 예시에서 Pydantic 스타일로 정의된 클래스를 사용하여 이메일 정보를 파싱해보겠습니다.  

#### 출력 파서를 사용한 경우

_Field 객체 안에 description 파라미터는 LLM이 정보를 추출할 때 이 설명을 보고 추출하기때문에 정확하고 명확해야 합니다._

In [23]:
class EmailSummury(BaseModel):  # BaseModel 기능을 상속받음
    sender_name: str = Field(description="발신자 이름")
    sender_department: str = Field(description="발신자 부서")
    sender_email: str = Field(description="발신자 이메일")
    subject: str = Field(description="메일 제목")
    summury: str = Field(description="메일 본문의 문제 요약")
    date: str = Field(description="문제 발생 시간(YYYY-MM-DD)")


parser = PydanticOutputParser(pydantic_object=EmailSummury)

In [24]:
format = parser.get_format_instructions()
print(format)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"sender_name": {"description": "발신자 이름", "title": "Sender Name", "type": "string"}, "sender_department": {"description": "발신자 부서", "title": "Sender Department", "type": "string"}, "sender_email": {"description": "발신자 이메일", "title": "Sender Email", "type": "string"}, "subject": {"description": "메일 제목", "title": "Subject", "type": "string"}, "summury": {"description": "메일 본문의 문제 요약", "title": "Summury", "type": "string"}, "date": {"description": "문제 발생 시간(YYYY-MM-DD)", "title": "Date", "type": "string"}}, "required": ["sender_nam

프롬프트를 정의합니다.

1. `question`: 유저의 질문을 받습니다.
2. `email_conversation`: 이메일 본문의 내용을 입력합니다.
3. `format`: 형식을 지정합니다.

In [25]:
prompt = PromptTemplate.from_template(
    """
    당신은 AI 어시드던트입니다. 아래 내용에 대한 질문을 할글로 대답해주세요.
    
    #Qustion:
    {question}
    
    #EMAIL CONVERSATION:
    {email_conversation}
    
    #FORMAT:
    {format}
    """
)

prompt = prompt.partial(format=format)  # 먼저 형식을 채움

In [26]:
chain = prompt | llm

In [27]:
response = chain.stream(
    {
        "email_conversation": email_conversation,
        "question": "다음의 이메일 내용중 중요한 내용을 추출해주세요.",
    }
)

# output은 JSON 형태로 출력
output = stream_response(response, return_output=True)

```json
{
  "sender_name": "박은빈",
  "sender_department": "기획팀",
  "sender_email": "siverempty@blockchainproject.com",
  "subject": "[수정 요청] 스마트 컨트랙트 민팅 조건 수정 필요",
  "summury": "스마트 컨트랙트의 민팅 조건이 TWAP 기준으로 설정되어 있어, 시장 가격이 충족되더라도 민팅이 차단되는 문제 발생. 사용자 불만 증가 및 프로젝트 신뢰도 저하 우려.",
  "date": "2024-11-18"
}
```

In [28]:
structured = parser.parse(output)  # parse 함수로 EmailSummury 객체로 변환
structured
# 객체로 변환되어 객체의 멤버변수에 접근 가능
# structured.sender_name
# structured.sender_department
# structured.sender_email
# structured.subject
# structured.summury
# structured.date

EmailSummury(sender_name='박은빈', sender_department='기획팀', sender_email='siverempty@blockchainproject.com', subject='[수정 요청] 스마트 컨트랙트 민팅 조건 수정 필요', summury='스마트 컨트랙트의 민팅 조건이 TWAP 기준으로 설정되어 있어, 시장 가격이 충족되더라도 민팅이 차단되는 문제 발생. 사용자 불만 증가 및 프로젝트 신뢰도 저하 우려.', date='2024-11-18')

#### Parser가 추가된 체인 생성

parser를 추가하여 체인을 구성할 수 있습니다.

In [30]:
# 출력 파서를 추가한 체인 구성
chain = prompt | llm | parser

response = chain.invoke(
    {
        "email_conversation": email_conversation,
        "question": "다음의 이메일 내용중 중요한 내용을 추출해주세요.",
    }
)

response

EmailSummury(sender_name='박은빈', sender_department='기획팀', sender_email='siverempty@blockchainproject.com', subject='[수정 요청] 스마트 컨트랙트 민팅 조건 수정 필요', summury='스마트 컨트랙트의 민팅 조건이 TWAP 기준으로 설정되어 있어, 시장 가격이 충족되더라도 민팅이 차단되는 문제 발생. 사용자 불만 증가 및 프로젝트 신뢰도 저하 우려.', date='2024-11-18')

#### Chain에 구조화 된 출력 추가

In [ ]:
response.sender_name  # chain 생성시에 parser를 추가하면 response 자체가 객체화 되어있음